In [1]:
import pandas as pd
import os
import time
import requests
import json
import csv
from tqdm import tqdm

import tweepy

import requests
import pandas as pd
import os
import collections
from langdetect import detect
import matplotlib.pyplot as plt

f = open("api-keys.json")
input=json.load(f)

In [2]:
import nltk
import string 
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

# GET STOPWORDS
from nltk.corpus import stopwords
stop_words=nltk.corpus.stopwords.words('english')

# INITALIZE STEMMER+LEMITZIZER+SIA
sia = SentimentIntensityAnalyzer()
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [3]:
# LOAD KEYS INTO API
consumer_key=input["consumer_key"]    
consumer_secret=input["consumer_secret"]    
access_token=input["access_token"]    
access_token_secret=input["access_token_secret"]    
bearer_token=input["bearer_token"]    


In [4]:
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)

api = tweepy.API(auth)
headers = {"Authorization": "Bearer {}".format(bearer_token)}
print(api)
print(api.verify_credentials().screen_name)

monk_digi


In [5]:
def search_more(query,tweets_num):
    num_tweets_collected=0
    searches=[]
    while num_tweets_collected<tweets_num:
        if len(searches)==0:
            search_results = api.search_tweets(query,lang="en", count=100)
        else:
            search_results = api.search_tweets(query,lang="en", count=100,max_id=max_id_next)
        num_tweets_collected+=len(search_results)
        max_id_next=int(search_results[-1]._json["id_str"])-1

        for i in range(len(search_results)):
            searches.append(search_results[i])
    return(searches)

In [7]:
def sh_to_list(searches):
    texts=[]
    for i in range(len(searches)):
        texts.append(searches[i]._json["text"])
    return texts


In [35]:
#acquire data from twitter

sh_free=search_more("freeway",3000)
sh_free_list=list(set(sh_to_list(sh_free)))
sh_mtr=search_more("metro",3000)
sh_mtr_list=list(set(sh_to_list(sh_mtr)))

Raw text data:

In [38]:
df_free.to_csv('tweet_freeway.csv',index=False)
df_metro.to_csv('tweet_metro.csv',index=False)

Clean the text data:

In [17]:
# clean the text data

def clean_string(text):
    #FILTER OUT UNWANTED CHAR
    new_text=""
    keep=" abcdefghijklmnopqrstuvwxyz0123456789"
    for character in text:
        if character.lower() in keep:
            new_text+=character.lower()
        else: 
            new_text+=" "
    text=new_text

    #FILTER OUT UNWANTED WORDS
    new_text=""
    for word in nltk.tokenize.word_tokenize(text):
        if word not in nltk.corpus.stopwords.words('english'):
            #lemmatize 
            tmp=lemmatizer.lemmatize(word)
            word=tmp
            if len(word)>1:
                if word in [".",",","!","?",":",";"]:
                    #remove the last space
                    new_text=new_text[0:-1]+word+" "
                else: #add a space
                    new_text+=word.lower()+" "
    text=new_text.strip()
    return text

In [18]:
for i in range(len(sh_free_list)):
    sh_free_list[i]=clean_string(sh_free_list[i])

In [19]:
for i in range(len(sh_mtr_list)):
    sh_mtr_list[i]=clean_string(sh_mtr_list[i])

In [20]:
tmp1=[]
for i in range(0,len(sh_free_list)):
    tmp1.append([sh_free_list[i],"freeway"])
df1=pd.DataFrame(tmp1)
df1=df1.rename(columns={0: "text", 1: "label"})
print(df1)

                                                   text    label
0     boenau rfsdfhsfbhwsfgb least ca expressway mig...  freeway
1     zacakamadu sudden urge go san jose talk cultur...  freeway
2           alinaaaziz yes girl freeway system wack lol  freeway
3     multi adsbx circling alert time sat dec 10 18 ...  freeway
4     ghettosmosh almost crashed fucking whip freewa...  freeway
...                                                 ...      ...
2652  20 east atlanta see neighborhood people priced...  freeway
2653  ever since first bmw 90 325is obsessed shadow ...  freeway
2654  cruising exact freeway 1930s german toward fas...  freeway
2655  briannawu even walking grocery store across st...  freeway
2656  congestion n1 inbound elevated freeway expect ...  freeway

[2657 rows x 2 columns]


In [26]:
tmp2=[]
for i in range(0,len(sh_mtr_list)):
    tmp2.append([sh_mtr_list[i],"metro"])
df2=pd.DataFrame(tmp2)
df2=df2.rename(columns={0: "text", 1: "label"})
print(df2)

                                                   text  label
0     congratulation district metro gold south honor...  metro
1     rt kumardprince1 ktrtrs ltmhyd best investment...  metro
2     rt chip vitaliy even going hell keep going ina...  metro
3     water stagnation 47th street 9th avenue ashokn...  metro
4     rt drdpgoel board nagpur metro hon ble prime m...  metro
...                                                 ...    ...
1360  rt veryrare metro boomin leaving studio making...  metro
1361  rt antsharwood interesting morning sydney weat...  metro
1362           hihyderabad abmandadapu metro station pl  metro
1363  debrobertsabc autopsy georgia2022election requ...  metro
1364  rt pugazh98 exclusive varisu sticker getting r...  metro

[1365 rows x 2 columns]


In [27]:
df=pd.concat([df1,df2])
print(df)
df.to_csv('free_mtr_text_data.csv',index=False)

                                                   text    label
0     boenau rfsdfhsfbhwsfgb least ca expressway mig...  freeway
1     zacakamadu sudden urge go san jose talk cultur...  freeway
2           alinaaaziz yes girl freeway system wack lol  freeway
3     multi adsbx circling alert time sat dec 10 18 ...  freeway
4     ghettosmosh almost crashed fucking whip freewa...  freeway
...                                                 ...      ...
1360  rt veryrare metro boomin leaving studio making...    metro
1361  rt antsharwood interesting morning sydney weat...    metro
1362           hihyderabad abmandadapu metro station pl    metro
1363  debrobertsabc autopsy georgia2022election requ...    metro
1364  rt pugazh98 exclusive varisu sticker getting r...    metro

[4022 rows x 2 columns]
